# Library

In [1]:
!pip install keras_ocr

     |████████████████████████████████| 166 kB 8.2 MB/s 
     |████████████████████████████████| 9.5 MB 13.0 MB/s 
     |████████████████████████████████| 803 kB 44.2 MB/s 
     |████████████████████████████████| 179 kB 49.1 MB/s 
     |████████████████████████████████| 131 kB 53.2 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.8.4-py3-none-any.whl size=148638 sha256=abab728252aa505fe177599a0a576fff9d802ae2a81015e972e7a4f1d042dbcd
  Stored in directory: /root/.cache/pip/wheels/a6/d8/34/518e393455207f8752d5777d1c174617d200cab5a5ed7bf2ad
  Created wheel for validators: filename=validators-0.15.0-py3-none-any.whl size=18371 sha256=aca6b97caaac6dc08bdb3529e2a2fe798788e981f72a871722e4f967cbe01f28
  Stored in directory: /root/.cache/pip/wheels/ff/a1/08/0650ea8df7821ecd87760d16213b53dd23403927a69a388268
Successfully built keras-ocr validators


In [2]:
!pip freeze > requirements.txt

In [3]:
import matplotlib.pyplot as plt
import keras_ocr

import numpy as np
import pandas as pd

from time import time
import re
import itertools
import multiprocessing
import gc

In [4]:
print('Numpy version:', np.__version__)
print('Pandas version:', pd.__version__)

Numpy version: 1.18.1
Pandas version: 1.0.3


# Dataset

In [5]:
!ls /kaggle/input

shopee-product-detection-student


In [6]:
df = pd.read_csv('/kaggle/input/shopee-product-detection-student/train.csv', dtype='object')
df['category'] = df['category'].apply(lambda c: str(c).zfill(2))
df.head()

,filename,category
0,45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg,03
1,f74d1a5fc2498bbbfa045c74e3cc333e.jpg,03
2,f6c172096818c5fab10ecae722840798.jpg,03
3,251ffd610399ac00fea7709c642676ee.jpg,03
4,73c7328b8eda399199fdedec6e4badaf.jpg,03


In [7]:
paths = []
for i in df.index:
    paths.append(f'/kaggle/input/shopee-product-detection-student/train/train/train/{df.loc[i, "category"]}/{df.loc[i, "filename"]}')

# Config OCR

In [8]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [9]:
# part 1 = 0:26347
# part 2 = 26347:52695
# part 3 = 52695:79042 (CURRENT)
# part 4 = 79042:TOTAL_IMAGES
TOTAL_IMAGES = len(paths)
BATCH_PREDICT = 5

START_INDEX = 52695
END_INDEX = 79042

list_texts = []

# Predict

In [10]:
for i in range(START_INDEX, END_INDEX, BATCH_PREDICT):
    try:
        if i + BATCH_PREDICT < END_INDEX:
            images = [keras_ocr.tools.read(p) for p in paths[i:i + BATCH_PREDICT]]
        else:
            images = [keras_ocr.tools.read(p) for p in paths[i:END_INDEX]]

        prediction_groups = pipeline.recognize(images)

        for x in range(len(prediction_groups)):
            texts = []

            for y in range(len(prediction_groups[x])):
                texts.append(prediction_groups[x][y][0])

            list_texts.append(texts)
        gc.collect()
    except:
        if i + BATCH_PREDICT < END_INDEX:
            for j in range(BATCH_PREDICT):
                list_texts.append([])
        else:
            for j in range(END_INDEX - i):
                list_texts.append([])

In [11]:
sr_text = pd.Series(list_texts)
sr_text

0                                               [daba, is]
1        [rashyacollestion, seiv, by, onyx, gade, qr, l...
2        [minion, selalu, menjadi, a, minion, papoy, mu...
3                                                       []
4                                               [un, fuud]
                               ...                        
26342    [m206, mlz, m205, m17b, m125, 22, miz1, m120, ...
26343                                                  [a]
26344    [nneexiadingdanis, nttn, sllshopees, twlxiaod,...
26345                                                   []
26346            [s, down, knock, stand, enema, hoo, grid]
Length: 26347, dtype: object

In [12]:
sr_text.to_csv('train2c.csv', index=False, header=False)